In [1]:
import os, sys
from pathlib import Path
import pandas as pd

sys.path.append(str(Path("..").resolve()))
from config.settings import DATA_DIR

all_positions_path = os.path.join(DATA_DIR, "All_Positions20260220.csv")

In [ ]:
all_positions = pd.read_csv(all_positions_path, header=0)
all_ = all_positions.copy()
text_cols = all_positions.select_dtypes(include=["object", "string"]).columns
all_positions[text_cols] = all_positions[text_cols].apply(
    lambda series: series.str.strip().str.lstrip("'").str.upper()
 )

In [ ]:
fund_parts = all_positions["FundID"].str.split("-", n=1, expand=True).reindex(columns=[0, 1])
all_positions["parent_fund"] = fund_parts[0].astype("string").str.strip()
all_positions["prime_broker"] = fund_parts[1].astype("string").str.strip()

basket_totals = (
    all_positions.loc[all_positions["prime_broker"].notna()]
    .groupby(["parent_fund", "SUPPLEMENTAL_ID_1"], as_index=False)["Basket Shares"]
    .sum()
)

In [ ]:
left_table = all_positions.loc[all_positions["prime_broker"].isna()].drop(
    columns=["Basket Shares", "parent_fund", "prime_broker"])

merged_df = left_table.merge(
    basket_totals,
    how="left",
    left_on=["FundID", "SUPPLEMENTAL_ID_1"],
    right_on=["parent_fund", "SUPPLEMENTAL_ID_1"],
    validate="many_to_one")

merged_df = merged_df.drop(columns=["parent_fund"]).fillna({"Basket Shares": 0})
merged_df.to_csv(f'{all_positions_path}', index=False)

In [5]:
print(merged_df.head())

  FundID ISIN CUSIP Ticker SEDOL Currency Country Restriction Trade Country  \
0   HRQ3  NaN   NaN   1AEM   NaN      CAD                   N            CA   
1   HRQY  NaN   NaN   1BCE   NaN      CAD                   N            CA   
2   HRCJ  NaN   NaN   1BMO   NaN      CAD                   N            CA   
3   HRCJ  NaN   NaN   1BNS   NaN      CAD                   N            CA   
4   HRQ2  NaN   NaN   1CCO   NaN      CAD                   N            CA   

   IDENTIFIER  CIN  ...  BASKET DATE  CORPORATE ACTION FACTOR  \
0         NaN  NaN  ...          NaN                      NaN   
1         NaN  NaN  ...          NaN                      NaN   
2         NaN  NaN  ...          NaN                      NaN   
3         NaN  NaN  ...          NaN                      NaN   
4         NaN  NaN  ...          NaN                      NaN   

   INTEREST FACTOR  SUPPRESSPRICE             SUPPLEMENTAL_ID_1  \
0              NaN            NaN  EO.CA.TO.1AEM..C275.20260306   
